In [1]:
from re import sub, search, findall
import os
import pandas as pd
import numpy as np
from glob import glob

In [2]:
lst = list(glob(os.path.join('c:/Users/Alex/D/phd/R/test', '*.txt')))

In [3]:
def remover(file_):
    file_ = ''.join(file_.split()).lower()
    stop_element = [',|от|менее|<|>|автоматическая|ручная|[(]|[)]'
                    '|меньше|меньшн|больше|более|менее|iultra|от|[wbc]']
    for elem in stop_element:
      if elem == ',':
        file_ = sub(elem, '.', file_)
      else:
        file_ = sub(elem, '', file_)
    return(file_)

In [4]:
def change_date_format(dt):
    """ на первом этапе переводим "нормальный формат" в формат через дефисы
     на втором этапе ищем вариант дд.мм.гг в формат через дефисы"""
    data_1 =  sub(r'(0?[1-9]|[12]\d|30|31)[.](0?[1-9]|1[0-2])[.](\d{4})',
                     '\\1-\\2-\\3', dt)
    return sub(r'(0?[1-9]|[12]\d|30|31)[.](0?[1-9]|1[0-2])[.](\d{2})',
                  '\\1-\\2-20\\3', data_1)

In [34]:
class DataExtractor:
    def __init__(self, list_of_epicrisis, test_dates = [], values = [], patients_id = [], patients_birthday = [], data_dct = {}):
        self.list_of_epicrisis = list_of_epicrisis
        self.test_dates = test_dates
        self.values = values
        self.patients_id = patients_id
        self.patients_birthday = patients_birthday
        self.data_dct = data_dct
   
        
    def fibrinogen(self):
        """fibrinogen

        params:
            list_of_epicrisis - list of txt files 
            test_dates - dates of fibrinogen test
            values - fibrinogen values
            patients_id - # case report
            patients_birthday

        returns:
            dataframe contains fibrinogen data
        """
        for doc in self.list_of_epicrisis:
          with open(doc, encoding='utf-8', errors='ignore') as file_:
            file_ = change_date_format(remover(file_.read()))
            file_ = sub(r'(\d{2}-\d{2}-\d{4})(гемостазиограмма)', r'\2\1', file_) #ставим дату после гемостазиограммы, чтобы проще было искать по одному шаблону.
            for date in findall(r'(?<=гемостазиограмма)(\d{2}-\d{2}-\d{4}|)', file_): #поиск дат, которые идут после гемост-ы.
              self.test_dates.append(date)
              id = search(r'\d+', file_) #ищем айди. находит самую первую дату в эпикризе. !!! есть эпикризы в которых первые цифры это номер отделения, типа онко отд. №2
              if id:
                    self.patients_id.append(id[0])
              birthday = search("(\d{2})-(\d{2})-(\d{4})", file_)
              if birthday:
                    self.patients_birthday.append(birthday[0])
            file_1 = sub(r'(гемостазиограмма)(\d{2}-\d{2}-\d{4})', r'\2\1', file_) #здесь наоборот ставим дату ПЕРЕД гемост-ой, чтобы она нам не мешала. тк вариант типа "гемостазиограмма27-09-2021:ачтв.." не даст результата.
            for value in findall(r'(гемостазиограмма.*?)(?=\d{2}-\d{2}-\d{4})', file_1): #задаем поиск строки от гемост-ы до даты другого исследования, при этом НЕ захватываются данные других иссл-ий.
              value = sub(r'(.*)(фибриноген)(\d.\d*)', r'\3\2\1', value) #в найденной строке меняем местами необходимые данные. новая строка имеет вид: 4,66фибриногенгемостазиограмма:ачтв24мно1,20...
              value = sub(r'(фибриноген|гемостазиограмма).*', '', value)
              if value: #удаляем все, что после фибриногена и/или гемост-ы включительно, оставляя лишь число. если его нет - пустая строка.
                  self.values.append(value)
              else:
                  self.values.append('NaN') #удаляем все, что после фибриногена и/или гемост-ы включительно, оставляя лишь число. если его нет - пустая строка.
               # а здесь почему-то не могу применить float/np.float. хотя во всем числах запятые заменены на точки. пишет - ValueError: could not convert string to float: 
        self.data_dct = {'ID':self.patients_id, 'Birthday':self.patients_birthday, 'dates':self.test_dates,'values':self.values}
        df = pd.DataFrame(self.data_dct)
        df = df.replace(r'', np.nan, regex=True)
        return(df)
    
    
    
    
    def C_reactive_protein(self):
        """C_reactive_protein

            params:
                list_of_epicrisis - list of txt files 
                test_dates - dates of C_reactive_protein test
                values - C_reactive_protein values
                patients_id - # case report
                patients_birthday

            returns:
                dataframe contains C_reactive_protein data
        """
        for doc in self.list_of_epicrisis:
          with open(doc, encoding='utf-8', errors='ignore') as file_:
            file_ = change_date_format(remover(file_.read()))
            file_ = sub(r'(\d{2}-\d{2}-\d{4})(биохимическоеисследованиекрови)', r'\2\1', file_) #ставим дату после гемостазиограммы, чтобы проще было искать по одному шаблону.
            for date in findall(r'(?<=биохимическоеисследованиекрови)(\d{2}-\d{2}-\d{4}|)', file_): #поиск дат, которые идут после гемост-ы.
              self.test_dates.append(date)
              id = search(r'\d+', file_) #ищем айди. находит самую первую дату в эпикризе. !!! есть эпикризы в которых первые цифры это номер отделения, типа онко отд. №2
              if id:
                    self.patients_id.append(id[0])
              birthday = search("(\d{2})-(\d{2})-(\d{4})", file_)
              if birthday:
                    self.patients_birthday.append(birthday[0])
            file_1 = sub(r'(биохимическоеисследованиекрови)(\d{2}-\d{2}-\d{4})', r'\2\1', file_) #здесь наоборот ставим дату ПЕРЕД гемост-ой, чтобы она нам не мешала. тк вариант типа "гемостазиограмма27-09-2021:ачтв.." не даст результата.
            for value in findall(r'(биохимическоеисследованиекрови.*?)(?=\d{2}-\d{2}-\d{4})', file_1): #задаем поиск строки от гемост-ы до даты другого исследования, при этом НЕ захватываются данные других иссл-ий.
              value = sub(r'(.*)(.-реактивныйбелок)(\d.\d*)', r'\3\2\1', value) #в найденной строке меняем местами необходимые данные. новая строка имеет вид: 4,66фибриногенгемостазиограмма:ачтв24мно1,20...
              value = sub(r'(.-реактивныйбелок|биохимическоеисследованиекрови).*', '', value)
              if value: #удаляем все, что после фибриногена и/или гемост-ы включительно, оставляя лишь число. если его нет - пустая строка.
                  self.values.append(value)
              else:
                  self.values.append('NaN') #удаляем все, что после фибриногена и/или гемост-ы включительно, оставляя лишь число. если его нет - пустая строка.
               # а здесь почему-то не могу применить float/np.float. хотя во всем числах запятые заменены на точки. пишет - ValueError: could not convert string to float: 
        self.data_dct = {'ID':self.patients_id, 'Birthday':self.patients_birthday, 'dates':self.test_dates,'values':self.values}
        df = pd.DataFrame(self.data_dct)
        df = df.replace(r'', np.nan, regex=True)
        return(df)
    
    
    def creatinine(self):
        """creatinine

            params:
                list_of_epicrisis - list of txt files 
                test_dates - dates of creatinine test
                values - creatinine values
                patients_id - # case report
                patients_birthday

            returns:
                dataframe contains creatinine data
        """
        for doc in self.list_of_epicrisis:
          with open(doc, encoding='utf-8', errors='ignore') as file_:
            file_ = change_date_format(remover(file_.read()))
            file_ = sub(r'(\d{2}-\d{2}-\d{4})(биохимическоеисследованиекрови)', r'\2\1', file_) #ставим дату после гемостазиограммы, чтобы проще было искать по одному шаблону.
            for date in findall(r'(?<=биохимическоеисследованиекрови)(\d{2}-\d{2}-\d{4}|)', file_): #поиск дат, которые идут после гемост-ы.
              self.test_dates.append(date)
              id = search(r'\d+', file_) #ищем айди. находит самую первую дату в эпикризе. !!! есть эпикризы в которых первые цифры это номер отделения, типа онко отд. №2
              if id:
                    self.patients_id.append(id[0])
              birthday = search("(\d{2})-(\d{2})-(\d{4})", file_)
              if birthday:
                    self.patients_birthday.append(birthday[0])
            file_1 = sub(r'(биохимическоеисследованиекрови)(\d{2}-\d{2}-\d{4})', r'\2\1', file_) #здесь наоборот ставим дату ПЕРЕД гемост-ой, чтобы она нам не мешала. тк вариант типа "гемостазиограмма27-09-2021:ачтв.." не даст результата.
            for value in findall(r'(биохимическоеисследованиекрови.*?)(?=\d{2}-\d{2}-\d{4})', file_1): #задаем поиск строки от гемост-ы до даты другого исследования, при этом НЕ захватываются данные других иссл-ий.
              value = sub(r'(.*)(кр.....ин)(\d.\d*)', r'\3\2\1', value) #в найденной строке меняем местами необходимые данные. новая строка имеет вид: 4,66фибриногенгемостазиограмма:ачтв24мно1,20...
              value = sub(r'(кр.....ин|биохимическоеисследованиекрови).*', '', value)
              if value: #удаляем все, что после фибриногена и/или гемост-ы включительно, оставляя лишь число. если его нет - пустая строка.
                  self.values.append(value)
              else:
                  self.values.append('NaN') #удаляем все, что после фибриногена и/или гемост-ы включительно, оставляя лишь число. если его нет - пустая строка.
               # а здесь почему-то не могу применить float/np.float. хотя во всем числах запятые заменены на точки. пишет - ValueError: could not convert string to float: 
        self.data_dct = {'ID':self.patients_id, 'Birthday':self.patients_birthday, 'dates':self.test_dates,'values':self.values}
        df = pd.DataFrame(self.data_dct)
        df = df.replace(r'', np.nan, regex=True)
        return(df)
    
    
    
    def AST(self):
        """AST

            params:
                list_of_epicrisis - list of txt files 
                test_dates - dates of AST test
                values - AST values
                patients_id - # case report
                patients_birthday

            returns:
                dataframe contains ALT data
        """
        for doc in self.list_of_epicrisis:
          with open(doc, encoding='utf-8', errors='ignore') as file_:
            file_ = change_date_format(remover(file_.read()))
            file_ = sub(r'(\d{2}-\d{2}-\d{4})(биохимическоеисследованиекрови)', r'\2\1', file_) #ставим дату после гемостазиограммы, чтобы проще было искать по одному шаблону.
            for date in findall(r'(?<=биохимическоеисследованиекрови)(\d{2}-\d{2}-\d{4}|)', file_): #поиск дат, которые идут после гемост-ы.
              self.test_dates.append(date)
              id = search(r'\d+', file_) #ищем айди. находит самую первую дату в эпикризе. !!! есть эпикризы в которых первые цифры это номер отделения, типа онко отд. №2
              if id:
                    self.patients_id.append(id[0])
              birthday = search("(\d{2})-(\d{2})-(\d{4})", file_)
              if birthday:
                    self.patients_birthday.append(birthday[0])
            file_1 = sub(r'(биохимическоеисследованиекрови)(\d{2}-\d{2}-\d{4})', r'\2\1', file_) #здесь наоборот ставим дату ПЕРЕД гемост-ой, чтобы она нам не мешала. тк вариант типа "гемостазиограмма27-09-2021:ачтв.." не даст результата.
            for value in findall(r'(биохимическоеисследованиекрови.*?)(?=\d{2}-\d{2}-\d{4})', file_1): #задаем поиск строки от гемост-ы до даты другого исследования, при этом НЕ захватываются данные других иссл-ий.
              value = sub(r'(.*)(аспартатаминрансфераза)(\d.\d*)', r'\3\2\1', value) #в найденной строке меняем местами необходимые данные. новая строка имеет вид: 4,66фибриногенгемостазиограмма:ачтв24мно1,20...
              value = sub(r'(аспартатаминрансфераза|биохимическоеисследованиекрови).*', '', value)
              if value: #удаляем все, что после фибриногена и/или гемост-ы включительно, оставляя лишь число. если его нет - пустая строка.
                  self.values.append(value)
              else:
                  self.values.append('NaN') #удаляем все, что после фибриногена и/или гемост-ы включительно, оставляя лишь число. если его нет - пустая строка.
               # а здесь почему-то не могу применить float/np.float. хотя во всем числах запятые заменены на точки. пишет - ValueError: could not convert string to float: 
        self.data_dct = {'ID':self.patients_id, 'Birthday':self.patients_birthday, 'dates':self.test_dates,'values':self.values}
        df = pd.DataFrame(self.data_dct)
        df = df.replace(r'', np.nan, regex=True)
        return(df)
    
    
    
    def ALT(self):
        """ALT

            params:
                list_of_epicrisis - list of txt files 
                test_dates - dates of ALT test
                values - ALT values
                patients_id - # case report
                patients_birthday

            returns:
                dataframe contains ALT data
        """
        for doc in self.list_of_epicrisis:
          with open(doc, encoding='utf-8', errors='ignore') as file_:
            file_ = change_date_format(remover(file_.read()))
            file_ = sub(r'(\d{2}-\d{2}-\d{4})(биохимическоеисследованиекрови)', r'\2\1', file_) #ставим дату после гемостазиограммы, чтобы проще было искать по одному шаблону.
            for date in findall(r'(?<=биохимическоеисследованиекрови)(\d{2}-\d{2}-\d{4}|)', file_): #поиск дат, которые идут после гемост-ы.
              self.test_dates.append(date)
              id = search(r'\d+', file_) #ищем айди. находит самую первую дату в эпикризе. !!! есть эпикризы в которых первые цифры это номер отделения, типа онко отд. №2
              if id:
                    self.patients_id.append(id[0])
              birthday = search("(\d{2})-(\d{2})-(\d{4})", file_)
              if birthday:
                    self.patients_birthday.append(birthday[0])
            file_1 = sub(r'(биохимическоеисследованиекрови)(\d{2}-\d{2}-\d{4})', r'\2\1', file_) #здесь наоборот ставим дату ПЕРЕД гемост-ой, чтобы она нам не мешала. тк вариант типа "гемостазиограмма27-09-2021:ачтв.." не даст результата.
            for value in findall(r'(биохимическоеисследованиекрови.*?)(?=\d{2}-\d{2}-\d{4})', file_1): #задаем поиск строки от гемост-ы до даты другого исследования, при этом НЕ захватываются данные других иссл-ий.
              value = sub(r'(.*)(аланинаминрансфераза)(\d.\d*)', r'\3\2\1', value) #в найденной строке меняем местами необходимые данные. новая строка имеет вид: 4,66фибриногенгемостазиограмма:ачтв24мно1,20...
              value = sub(r'(аланинаминрансфераза|биохимическоеисследованиекрови).*', '', value)
              if value: #удаляем все, что после фибриногена и/или гемост-ы включительно, оставляя лишь число. если его нет - пустая строка.
                  self.values.append(value)
              else:
                  self.values.append('NaN') #удаляем все, что после фибриногена и/или гемост-ы включительно, оставляя лишь число. если его нет - пустая строка.
               # а здесь почему-то не могу применить float/np.float. хотя во всем числах запятые заменены на точки. пишет - ValueError: could not convert string to float: 
        self.data_dct = {'ID':self.patients_id, 'Birthday':self.patients_birthday, 'dates':self.test_dates,'values':self.values}
        df = pd.DataFrame(self.data_dct)
        df = df.replace(r'', np.nan, regex=True)
        return(df)

In [35]:
s = DataExtractor(lst)

In [36]:
df = s.ALT()

In [37]:
df

,ID,Birthday,dates,values
0,3113,08-06-1947,02-12-2020,29
1,3113,08-06-1947,04-12-2020,NaN
2,3113,08-06-1947,10-12-2020,83
3,3113,08-06-1947,14-12-2020,1.5
4,3113,08-06-1947,17-12-2020,153
...,...,...,...,...
159,27619,20-12-1974,02-12-2020,43
160,27619,20-12-1974,07-12-2020,51
161,27627,19-03-1957,02-12-2020,27
162,27627,19-03-1957,07-12-2020,70


In [175]:
def crp(list_of_epicrisis, 
               test_dates = [],
               values = [],
               patients_id = [],
               patients_birthday = []):
    """fibrinogen
    
    params:
        list_of_epicrisis - list of txt files 
        test_dates - dates of fibrinogen test
        values - fibrinogen values
        patients_id - # case report
        patients_birthday
        
    returns:
        dataframe contains fibrinogen data
    """
for doc in lst:
  with open(doc, encoding='utf-8', errors='ignore') as file_:
    file_ = change_date_format(remover(file_.read()))
    file_ = sub(r'(\d{2}-\d{2}-\d{4})(общийанализкрови)', r'\2\1', file_) #ставим дату после гемостазиограммы, чтобы проще было искать по одному шаблону.
    for date in findall(r'(?<=общийанализкрови)(\d{2}-\d{2}-\d{4}|)', file_): #поиск дат, которые идут после гемост-ы.
      dates.append(date)
      id = search(r'\d+', file_) #ищем айди. находит самую первую дату в эпикризе. !!! есть эпикризы в которых первые цифры это номер отделения, типа онко отд. №2
      if id:
        id_list.append(id[0])
    file_1 = sub(r'(общийанализкрови)(\d{2}-\d{2}-\d{4})', r'\2\1', file_) #здесь наоборот ставим дату ПЕРЕД гемост-ой, чтобы она нам не мешала. тк вариант типа "гемостазиограмма27-09-2021:ачтв.." не даст результата.
    for value in findall(r'(общийанализкрови.*?)(?=\d{2}-\d{2}-\d{4})', file_1): #задаем поиск строки от гемост-ы до даты другого исследования, при этом НЕ захватываются данные других иссл-ий.
      value = sub(r'(.*)(лейкоциты)(\d.\d*)', r'\3\2\1', value) #в найденной строке меняем местами необходимые данные. новая строка имеет вид: 4,66фибриногенгемостазиограмма:ачтв24мно1,20...
      value = sub(r'(лейкоциты|общийанализкрови).*', '', value)
      if value: #удаляем все, что после фибриногена и/или гемост-ы включительно, оставляя лишь число. если его нет - пустая строка.
          values.append(value)
      else:
          values.append('NaN') #удаляем все, что после фибриногена и/или гемост-ы включительно, оставляя лишь число. если его нет - пустая строка.
       # а здесь почему-то не могу применить float/np.float. хотя во всем числах запятые заменены на точки. пишет - ValueError: could not convert string to float:  
    data_dct = {'ID':patients_id, 'Birthday':patients_birthday, 'dates':test_dates,'values':values}
    df = pd.DataFrame(data_dct)
    df = df.replace(r'', np.nan, regex=True)
    return(df)

In [176]:
crp(lst)

,ID,Birthday,dates,values


In [177]:
print(len(test_dates),       len(values),               len(patients_id) ,
               len(patients_birthday))

97 0 1164 97


In [115]:
df = f.fibrinogen(list_of_epicrisis = lst)

In [116]:
df

""


In [169]:
dates = []
values = []
id_list = []

In [170]:
for doc in lst:
  with open(doc, encoding='utf-8', errors='ignore') as file_:
    file_ = change_date_format(remover(file_.read()))
    file_ = sub(r'(\d{2}-\d{2}-\d{4})(биохимическоеисследованиекрови)', r'\2\1', file_) #ставим дату после гемостазиограммы, чтобы проще было искать по одному шаблону.
    for date in findall(r'(?<=биохимическоеисследованиекрови)(\d{2}-\d{2}-\d{4}|)', file_): #поиск дат, которые идут после гемост-ы.
      dates.append(date)
      id = search(r'\d+', file_) #ищем айди. находит самую первую дату в эпикризе. !!! есть эпикризы в которых первые цифры это номер отделения, типа онко отд. №2
      if id:
        id_list.append(id[0])
    file_1 = sub(r'(биохимическоеисследованиекрови)(\d{2}-\d{2}-\d{4})', r'\2\1', file_) #здесь наоборот ставим дату ПЕРЕД гемост-ой, чтобы она нам не мешала. тк вариант типа "гемостазиограмма27-09-2021:ачтв.." не даст результата.
    for value in findall(r'(биохимическоеисследованиекрови.*?)(?=\d{2}-\d{2}-\d{4})', file_1): #задаем поиск строки от гемост-ы до даты другого исследования, при этом НЕ захватываются данные других иссл-ий.
      value = sub(r'(.*)(.-реактивныйбелок)(\d.\d*)', r'\3\2\1', value) #в найденной строке меняем местами необходимые данные. новая строка имеет вид: 4,66фибриногенгемостазиограмма:ачтв24мно1,20...
      value = sub(r'(.-реактивныйбелок|биохимическоеисследованиекрови).*', '', value)
      if value: #удаляем все, что после фибриногена и/или гемост-ы включительно, оставляя лишь число. если его нет - пустая строка.
          values.append(value)
      else:
          values.append('NaN') #удаляем все, что после фибриногена и/или гемост-ы включительно, оставляя лишь число. если его нет - пустая строка.
       # а здесь почему-то не могу применить float/np.float. хотя во всем числах запятые заменены на точки. пишет - ValueError: could not convert string to float: 

In [171]:
print(len(id_list), len(dates), len(values))

0 0 0


In [25]:
test_dates = []
values = []
patients_id = []
patients_birthday = []

In [ ]:
print(len(test_dates),       len(values),               len(patients_id) ,
               len(patients_birthday))

In [26]:
for doc in lst:
      with open(doc, encoding='utf-8', errors='ignore') as file_:
        file_ = change_date_format(remover(file_.read()))
        file_ = sub(r'(\d{2}-\d{2}-\d{4})(гемостазиограмма)', r'\2\1', file_) #ставим дату после гемостазиограммы, чтобы проще было искать по одному шаблону.
        for date in findall(r'(?<=гемостазиограмма)(\d{2}-\d{2}-\d{4}|)', file_): #поиск дат, которые идут после гемост-ы.
          test_dates.append(date)
          id = search(r'\d+', file_) #ищем айди. находит самую первую дату в эпикризе. !!! есть эпикризы в которых первые цифры это номер отделения, типа онко отд. №2
          if id:
                patients_id.append(id[0])
          birthday = search("(\d{2})-(\d{2})-(\d{4})", file_)
          if birthday:
                patients_birthday.append(birthday[0])
        file_1 = sub(r'(гемостазиограмма)(\d{2}-\d{2}-\d{4})', r'\2\1', file_) #здесь наоборот ставим дату ПЕРЕД гемост-ой, чтобы она нам не мешала. тк вариант типа "гемостазиограмма27-09-2021:ачтв.." не даст результата.
        for value in findall(r'(гемостазиограмма.*?)(?=\d{2}-\d{2}-\d{4})', file_1): #задаем поиск строки от гемост-ы до даты другого исследования, при этом НЕ захватываются данные других иссл-ий.
          value = sub(r'(.*)(фибриноген)(\d.\d*)', r'\3\2\1', value) #в найденной строке меняем местами необходимые данные. новая строка имеет вид: 4,66фибриногенгемостазиограмма:ачтв24мно1,20...
          value = sub(r'(фибриноген|гемостазиограмма).*', '', value)
          if value: #удаляем все, что после фибриногена и/или гемост-ы включительно, оставляя лишь число. если его нет - пустая строка.
              values.append(value)
          else:
              values.append('NaN')

In [163]:
print(len(test_dates),       len(values),               len(patients_id) ,
               len(patients_birthday))

97 97 1164 97


In [8]:
data_dct = {'ID':id_list, 'dates':dates,'values':values}
df = pd.DataFrame(data_dct)
df.replace(r'', np.nan, regex=True)

,ID,dates,values
0,3113,02-12-2020,3.96
1,3113,14-12-2020,3.98
2,3131,04-12-2020,4.4
3,3133,04-12-2020,4.4
4,3133,05-12-2020,NaN
...,...,...,...
92,27529,08-12-2020,72
93,27529,12-12-2020,53
94,27619,02-12-2020,7.72
95,27627,02-12-2020,6.33


In [168]:
print(len(id_list), len(dates), len(values))

0 0 0


In [30]:
data_dct = {'ID':id_list, 'dates':dates,'values':values}
df = pd.DataFrame(data_dct)
df.replace(r'', np.nan, regex=True)

,ID,dates,values
0,3113,02-12-2020,126
1,3113,04-12-2020,NaN
2,3113,10-12-2020,33
3,3113,14-12-2020,39
4,3113,17-12-2020,36
...,...,...,...
159,27619,02-12-2020,38
160,27619,07-12-2020,2.4
161,27627,02-12-2020,96
162,27627,07-12-2020,8.7


In [116]:
dates = []
values = []
id_list = []

In [105]:
file_ = change_date_format(remover("общийанализкрови04-12-2020лейкоциты13.8 10*9/л ( 4-9 )10*9/л; Эритроциты 4.27; Гемоглобин 115 г/л; СОЭ 43 мм/час; Гематокрит 35.8 %; Тромбоциты 562; Базофилы 1 %; Эозинофилы 2 %; Нейтрофилы палочкоядерные 2 %; Нейтрофилы сегментоядерные 75 %; Лимфоциты 15 %; Моноциты 5 %; "))
file_ = sub(r'(\d{2}-\d{2}-\d{4})(общийанализкрови)', r'\2\1', file_) #

In [106]:
file_

'общийанализкрови04-12-2020лейкоциты13.810*9/л4-910*9/л;эритроциты4.27;гемоглобин115г/л;соэ43мм/час;гематокрит35.8%;тромбоциты562;базофилы1%;эозинофилы2%;нейтрофилыпалочкоядерные2%;нейтрофилысегментоядерные75%;лимфоциты15%;моноциты5%;'

In [108]:
findall(r'(общийанализкрови.*?)(?=\d{2}-\d{2}-\d{4})', file_)

['общийанализкрови']

In [111]:
#file_1 = sub(r'(общийанализкрови)(\d{2}-\d{2}-\d{4})', r'\2\1', file_)
v = findall(r'(общийанализкрови.*?)(?=\d{2}-\d{2}-\d{4})', file_)
sub(r'(.*)(лейкоциты)(\d.\d*)', r'\3\2\1', file_1)

'13лейкоциты04-12-2020общийанализкрови:.810*9/л4-910*9/л;эритроциты4.27;гемоглобин115г/л;соэ43мм/час;гематокрит35.8%;тромбоциты562;базофилы1%;эозинофилы2%;нейтрофилыпалочкоядерные2%;нейтрофилысегментоядерные75%;лимфоциты15%;моноциты5%;'

In [117]:
file_ = change_date_format(remover("общийанализкрови04-12-2020лейкоциты13.8 10*9/л ( 4-9 )10*9/л; Эритроциты 4.27; Гемоглобин 115 г/л; СОЭ 43 мм/час; Гематокрит 35.8 %; Тромбоциты 562; Базофилы 1 %; Эозинофилы 2 %; Нейтрофилы палочкоядерные 2 %; Нейтрофилы сегментоядерные 75 %; Лимфоциты 15 %; Моноциты 5 %; "))
file_ = sub(r'(\d{2}-\d{2}-\d{4})(общийанализкрови)', r'\2\1', file_) #ставим дату после гемостазиограммы, чтобы проще было искать по одному шаблону.
for date in findall(r'(?<=общийанализкрови)(\d{2}-\d{2}-\d{4}|)', file_): #поиск дат, которые идут после гемост-ы.
    dates.append(date)
    id = search(r'\d+', file_) #ищем айди. находит самую первую дату в эпикризе. !!! есть эпикризы в которых первые цифры это номер отделения, типа онко отд. №2
    if id:
        id_list.append(id[0])
file_1 = sub(r'(общийанализкрови)(\d{2}-\d{2}-\d{4})', r'\2\1', file_) #здесь наоборот ставим дату ПЕРЕД гемост-ой, чтобы она нам не мешала. тк вариант типа "гемостазиограмма27-09-2021:ачтв.." не даст результата.
for value in findall(r'(общийанализкрови.*?)(?=\d{2}-\d{2}-\d{4})', file_1): #задаем поиск строки от гемост-ы до даты другого исследования, при этом НЕ захватываются данные других иссл-ий.
    value = sub(r'(.*)(лейкоциты)(\d.\d*)', r'\3\2\1', file_) #в найденной строке меняем местами необходимые данные. новая строка имеет вид: 4,66фибриногенгемостазиограмма:ачтв24мно1,20...
    value = sub(r'(лейкоциты|общийанализкрови).*', '', file_1)
    if value: #удаляем все, что после фибриногена и/или гемост-ы включительно, оставляя лишь число. если его нет - пустая строка.
        values.append(value)
    else:
        values.append('NaN') #удаляем все, что после фибриногена и/или гемост-ы включительно, оставляя лишь число. если его нет - пустая строка.
       # а здесь почему-то не могу применить float/np.float. хотя во всем числах запятые заменены на точки. пишет - ValueError: could not convert string to float: 

In [119]:
values

[]

In [118]:
data_dct = {'ID':id_list, 'dates':dates,'values':values}
df = pd.DataFrame(data_dct)
df.replace(r'', np.nan, regex=True)

ValueError: All arrays must be of the same length